<a href="https://colab.research.google.com/github/javilledo/machine-learning/blob/master/notebooks/t05_02_logistic_regression_implementaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Implementación del método de la máxima verosimilitud para la regresión logística

###Definir la función de entorno 

$L(b)=\prod_{i=1}^n P_i^{y_i} \cdot (1-P_i)^{y_i}$

In [62]:
def likelihold(y,pi):
  import numpy as np
  total_sum = 1
  sum_in = list(range(1,len(y)+1))
  for i in range(len(y)):
    sum_in[i]=np.where(y[i]==1, pi[i], 1-pi[i])
    total_sum = total_sum * sum_in[i]
  return total_sum 

###Calcular las probabilidades para cada observación

$P_i = P(x_i) = \frac{1}{1+e^{-\sum_{j=0}^k\beta_j \cdot x_{ij}}}$, donde $\alpha = \beta_0$


In [63]:
def logitprobs(X,beta):
  import numpy as np
  n_rows = np.shape(X)[0]
  n_cols = np.shape(X)[1]
  pi = list(range(1, n_rows+1))
  exponente = list(range(1, n_rows+1))
  for i in range(n_rows):
    exponente[i]= 0
    for j in range(n_cols):
      expon = X[i][j] * beta[j]
      exponente[i] = expon + exponente[i]
    with np.errstate(divide='ignore', invalid='ignore'):
      pi[i] = 1/(1+np.exp(-exponente[i]))
  return pi

###Calcular la matriz diagonal $W$

$W=diag(P_i \cdot(1-P_i))_{i=1}^n$

In [64]:
def findW(pi):
  import numpy as np
  n = len(pi)
  W = np.zeros(n*n).reshape(n,n)
  for i in range(n):
    print(i)
    W[i,i] = pi[i] * (1 - pi[i])
    W[i,i].astype(float)
  return W

###Obtener la solución de la función logística

$\beta_{n+1} = \beta_n - \frac{f(\beta_n)}{f'(\beta_n)}$

$f(\beta) = X(Y-P)$

$f'(\beta) = XWX^T$

In [65]:
def logistics(X, Y, limit):
  import numpy as np
  from numpy import linalg
  n_rows = np.shape(X)[0]
  bias = np.ones(n_rows).reshape(n_rows,1)
  X_new = np.append(X,bias, axis=1)
  n_cols = np.shape(X_new)[1]
  beta = np.zeros(n_cols).reshape(n_cols,1)
  root_dif = np.array(range(1,n_cols+1)).reshape(n_cols,1)
  iter_i = 10000
  while(iter_i >  limit):
    print('iter_i: ' + str(iter_i) + ", limit: " + str(limit))
    pi = logitprobs(X_new, beta)
    print('pi: ' + str(pi))
    W = findW(pi)
    print('W:' + str(W))
    num = (np.transpose(np.matrix(X_new)) * np.matrix(Y - np.transpose(pi)).transpose())
    den = (np.matrix(np.transpose(X_new)) * np.matrix(W) * np.matrix(X_new))
    root_dif = np.array(linalg.inv(den) * num)
    beta = beta + root_dif
    print('beta: ' + str(beta))
    iter_i = np.sum(root_dif * root_dif)
    ll = likelihold(Y, pi)
  return beta

###Comprobación experimental

In [66]:
import numpy as np

In [67]:
X = np.array(range(10)).reshape(10,1)
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [68]:
Y = [0,0,0,0,1,0,1,0,1,1]
Y

[0, 0, 0, 0, 1, 0, 1, 0, 1, 1]

In [69]:
bias = np.ones(10).reshape(10,1)
X_new = np.append(X,bias, axis = 1)
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [70]:
a = logistics(X,Y,0.00001)
a

iter_i: 10000, limit: 1e-05
pi: [array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
0
1
2
3
4
5
6
7
8
9
W:[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
beta: [[ 0.43636364]
 [-2.36363636]]
iter_i: 5.777190082644626, limit: 1e-05
pi: [array([0.08598797]), array([0.12705276]), array([0.18378532]), array([0.2583532]), array([0.35019508]), array([0.45467026]), array([0.56329497]), array([0.66616913]), array([0.75533524]), arra

array([[ 0.66220827],
       [-3.69557172]])

Hay que tener en cuenta que, como se ha añadido al final de la matriz X_new el array de unos, en los resultados de las betas será el último de los valores el término independiente(alpha). Así, el resultado de nuestro modelo es el siguiente:

$Y(x) = \frac{1}{1+e^{3.69557172 - 0.66220827 \cdot x}}$

In [71]:
ll = likelihold(Y,logitprobs(X,a))
ll

array([1.32622426e-06])

###Con el paquete stats de python

In [74]:
import statsmodels.api as sm

In [75]:
logitmodel = sm.Logit(Y,X_new)
result = logitmodel.fit()
result

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [76]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   10
Model:                          Logit   Df Residuals:                        8
Method:                           MLE   Df Model:                            1
Date:                Sat, 29 Aug 2020   Pseudo R-squ.:                  0.3596
Time:                        07:51:34   Log-Likelihood:                -4.3101
converged:                       True   LL-Null:                       -6.7301
Covariance Type:            nonrobust   LLR p-value:                   0.02781
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.6622      0.400      1.655      0.098      -0.122       1.446
const         -3.6956      2.289     -1.615      0.106      -8.182       0.791
